### SparkSQL

In [0]:
%python
directory = '/Volumes/workspace/default/sample_data/ManishKumar/'

from pyspark.sql.functions import *
from pyspark.sql.types import *

import pandas as pd
import json

# sample json string for schema inference
json_string = '{"user_id":"0001","first_name":"Akshay","listings": [{"listing_id":"847254","place": {"Area":"Naupada","City":"Thane"},"description":"apartment","services":[{"service_id":"BG111","service_type":"CookingGas","service_provider":"BharatGas"},{"service_id":"MV111","service_type":"Electricity","service_provider":"Mahavitaran"}]},{"listing_id":"435543","place": {"Area":"ShivajiNagar","City":"Pune"},"description":"vila","services":[{"service_id":"HG111","service_type":"CookingGas","service_provider":"HidustanGas"},{"service_id":"RL111","service_type":"Electricity","service_provider":"Reliance"}]}]}'

json_schema = 'STRUCT<first_name: STRING, listings: ARRAY<STRUCT<description: STRING, listing_id: STRING, place: STRUCT<Area: STRING, City: STRING>, services: ARRAY<STRUCT<service_id: STRING, service_provider: STRING, service_type: STRING>>>>, user_id: STRING>'

In [0]:
%python

spark.sql(f"""
          create or replace temporary view vw_json_data_with_schema_identified as
        select  *
                ,_metadata.file_modification_time AS file_modification_time -- file modification time
                ,_metadata.file_name AS source_file -- Ingestion data source file name
                ,current_timestamp() as ingestion_time  -- Ingestion timestamp
        from    read_files('{directory}json_data.json'
                ,format=>'json'
                ,schema=>schema_of_json('{json_string}')
                ,multiLine=>'true'
        )
        """)

# you can also use schema=>json_schema ... (which is declared above, if schema is known already)

# if you want to include a "rescued_data_column", add below line to read_files() clause:
# rescueddatacolumn => '_rescued_data'

spark.sql(f"""select * from vw_json_data_with_schema_identified""").display()

In [0]:
create or replace temporary view vw_full_data as
select  * except (services_exploded)
        ,services_exploded.service_id as service_id
        ,services_exploded.service_provider as service_provider
        ,services_exploded.service_type as service_type
from    (
        select  * except (listings_exploded)
                ,listings_exploded.listing_id as Property_id
                ,listings_exploded.description as description
                ,listings_exploded.place.Area as place_Area
                ,listings_exploded.place.City as place_City
                ,explode(listings_exploded.services) as services_exploded
        from    (
                select * except(listings)
                        ,explode(listings) as listings_exploded
                from  vw_json_data_with_schema_identified
                )t1
        )t2
order by user_id, Property_id, service_id;

select user_id,first_name,Property_id,description,place_Area,place_City,Service_id,Service_type,      
      Service_provider
from vw_full_data

### Using JSON

In [0]:
use workspace.default;
drop table if exists json_data;
create table json_data
using json
options(
   path "Volumes/workspace/default/sample_data/ManishKumar/json_data.json"
   , inferSchema "true"
   , mode "PERMISSIVE"
)

### JSON.path

In [0]:
CREATE OR REPLACE TEMPORARY VIEW customers AS
SELECT * FROM JSON.`s3://bookstorerawdata/dataset/ASSOCIATE/customers-json/`;

select * from customers limit 10;

In [0]:
/*
"schema_of_json" --> derives schema details (DDL format) from nested JSON data provided.
"from_json" --> converts JSON string "profile" into "struct" data type with help of schema derived in step1.
*/

CREATE OR REPLACE TEMPORARY VIEW parsed_customers AS
SELECT  customer_id
        ,profile
        ,profile_struct
from    (
        SELECT  customer_id,profile
                ,from_json(
                  profile
                  ,schema_of_json('{"first_name":"Thomas","last_name":"Lane","gender":"Male","address":{"street":"06 Boulevard Victor Hugo","city":"Paris","country":"France"}}')
                  ) AS profile_struct
        FROM customers
        );

select * from parsed_customers limit 10;



In [0]:

DESCRIBE customers; --nested data stored as plain string (profile).
DESCRIBE parsed_customers; --nested data stored as struct data type (profile & address).

-- use colon (:) to reference fields in a nested json "string". 
-- But, you cannot use * operator to reference "all" fields; instead you need to reference each field individually by its name.

SELECT customer_id,profile
FROM customers;
--OR
SELECT customer_id
,profile:gender
,profile:first_name
,profile:last_name
,profile:address:street
,profile:address:city
,profile:address:country 
FROM customers;

-- use dot (.) to reference fields in a "struct" data type. 
-- Here, you can use * operator to reference "all" fields.
SELECT customer_id,profile_struct
FROM parsed_customers;
--OR
SELECT customer_id,profile_struct.*
FROM parsed_customers;
--OR
SELECT customer_id
,profile_struct.gender
,profile_struct.first_name
,profile_struct.last_name
,profile_struct.address.street
,profile_struct.address.city
,profile_struct.address.country
FROM parsed_customers;